# Getting and Knowing your Data

Read raw dataset:

In [1]:
// input path
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val schema = StructType(
    Array(
        StructField("order_id", IntegerType, false),
        StructField("quantity", IntegerType, false),
        StructField("item_name", StringType, false),
        StructField("choice_description", StringType, true),
        StructField("item_price", StringType, false)
    )
)

val path = new java.io.File("datasets/chipotle.tsv").getCanonicalPath()
val rawDF = spark.read
    .option("header", "true")
    .option("delimiter", "\t")
    .schema(schema)
    .csv(path)



Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.5:4040
SparkContext available as 'sc' (version = 3.1.1-amzn-0, master = local[*], app id = local-1647680154164)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(order_id,IntegerType,false), StructField(quantity,IntegerType,false), StructField(item_name,StringType,false), StructField(choice_description,StringType,true), StructField(item_price,StringType,false))
path: String = /home/kuba/projects/pd-exercises-scala-spark/datasets/chipotle.tsv
rawDF: org.apache.spark.sql.DataFrame = [order_id: int, quantity: int ... 3 more fields]


Inspect data:

In [2]:
rawDF.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows



### What is the number of observations in the dataset?

In [3]:
rawDF.count()

res1: Long = 4622


### What is the number of columns in the dataset?

In [4]:
rawDF.columns.size

res2: Int = 5


### Print the name of all the columns.

In [5]:
rawDF.columns.foreach(println)

order_id
quantity
item_name
choice_description
item_price


### Which was the most-ordered item?

In [6]:
rawDF
    .select("quantity", "item_name")
    .groupBy("item_name")
    .agg(sum("quantity").alias("orders_sum"))
    .orderBy($"orders_sum".desc)
    .show(1)

+------------+----------+
|   item_name|orders_sum|
+------------+----------+
|Chicken Bowl|       761|
+------------+----------+
only showing top 1 row



### What was the most ordered item in the choice_description column?

In [19]:
rawDF
    .filter(!$"choice_description".contains("NULL"))
    .select("quantity", "choice_description")
    .groupBy("choice_description")
    .agg(sum("quantity").alias("orders_sum"))
    .orderBy($"orders_sum".desc)
    .show(1)

+------------------+----------+
|choice_description|orders_sum|
+------------------+----------+
|       [Diet Coke]|       159|
+------------------+----------+
only showing top 1 row



### How many items were orderd in total?

In [26]:
rawDF
    .select(sum("quantity").as("items_ordered_total"))
    .show()

+-------------------+
|items_ordered_total|
+-------------------+
|               4972|
+-------------------+



### Turn the item price into a float

In [49]:
val DF = rawDF
            .withColumn("item_price", regexp_replace(col("item_price"), "\\$", "").cast("Float"))

DF.show(5)
DF.printSchema()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|      2.39|
|       1|       1|                Izze|        [Clementine]|      3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39|
|       1|       1|Chips and Tomatil...|                NULL|      2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)



DF: org.apache.spark.sql.DataFrame = [order_id: int, quantity: int ... 3 more fields]


### How much was the revenue for the period in the dataset?

In [54]:
DF
    .select(round(sum($"item_price" * $"quantity"), 2).as("revenue"))
    .show()

+--------+
| revenue|
+--------+
|39237.02|
+--------+

